In [209]:
import tensorflow as tf
import params
import yamnet as yamnet_model
import json
import numpy as np

graph = tf.Graph()
# with graph.as_default():
yamnet = yamnet_model.yamnet_frames_model(params)
yamnet.load_weights("yamnet.h5")a

weights = yamnet.get_weights()
yamnet.summary()

Model: "yamnet_frames"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(1, None)]          0                                            
__________________________________________________________________________________________________
tf_op_layer_Squeeze_17 (TensorF [(None,)]            0           input_18[0][0]                   
__________________________________________________________________________________________________
tf_op_layer_stft_17/frame/Shape [(1,)]               0           tf_op_layer_Squeeze_17[0][0]     
__________________________________________________________________________________________________
tf_op_layer_stft_17/frame/split [(0,), (1,), (0,)]   0           tf_op_layer_stft_17/frame/Shape[0
______________________________________________________________________________________

In [290]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense


np.array(yamnet.get_layer("layer1/conv").get_weights()).shape



[<tf.Variable 'layer1/conv/bn_17/beta:0' shape=(32,) dtype=float32, numpy=
 array([ 0.04602636,  0.17091215, -0.02295654,  0.10236555,  0.13807301,
         0.01801817,  0.20645505,  0.1786242 ,  0.16670331,  0.18296945,
         0.01315918,  0.21252275,  0.24482024,  0.32340148,  0.0065095 ,
         0.04651999,  0.06499898, -0.13706996,  0.08591899,  0.02588778,
         0.2138525 ,  0.3123125 ,  0.2290062 ,  0.03190372,  0.23353843,
         0.13032568,  0.2188657 ,  0.14596836,  0.06833133,  0.07401527,
         0.06176487,  0.09753159], dtype=float32)>,
 <tf.Variable 'layer1/conv/bn_17/moving_mean:0' shape=(32,) dtype=float32, numpy=
 array([ 0.00330193, -0.01062203, -0.00362168, -0.0045226 , -0.00640222,
         0.07545111, -0.00204938, -0.00136688,  0.00116968, -0.01324444,
         0.06087347, -0.00146356, -0.00252752, -0.00501002,  0.06196222,
         0.009484  , -0.07466652,  0.00101532,  0.03975811,  0.06022405,
         0.0013755 , -0.00266118,  0.00204467,  0.01738453,  

In [298]:
import torch.nn as nn

class MobileNetV1(nn.Module):
    def __init__(self, dims, input_channels, embedding_size=1000):
        super(MobileNetV1, self).__init__()
        assert 1 <= dims <= 3
        
        # Initialise network for correct dimension size
        self.dims = dims

        opt_conv = [None, nn.Conv1d, nn.Conv2d, nn.Conv3d]
        opt_bn = [None, nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d]
        opt_pool = [None, nn.AvgPool1d, nn.AvgPool2d, nn.AvgPool3d]

        Conv = opt_conv[dims]
        BatchNorm = opt_bn[dims]
        self.AvgPool = opt_pool[dims]

        def _conv(inp, oup, stride):
            return nn.Sequential(
                Conv(inp, oup, 3, stride, 1, bias=False),
                BatchNorm(oup, eps=1e-4),
                nn.ReLU(inplace=True))

        def _separable_conv(inp, oup, stride):
            return nn.Sequential(
                # Depthwise conv block
                Conv(inp, inp, 3, stride, 1, groups=inp, bias=False),
                BatchNorm(inp),
                nn.ReLU(inplace=True),

                # Pointwise conv block
                Conv(inp, oup, 1, 1, 0, bias=False),
                BatchNorm(oup),
                nn.ReLU(inplace=True))

        self.features = nn.Sequential(
            _conv(input_channels,   32, 2),
            _separable_conv(  32,   64, 1),
            _separable_conv(  64,  128, 2),
            _separable_conv( 128,  128, 1),
            _separable_conv( 128,  256, 2),
            _separable_conv( 256,  256, 1),
            _separable_conv( 256,  512, 2),
            _separable_conv( 512,  512, 1),
            _separable_conv( 512,  512, 1),
            _separable_conv( 512,  512, 1),
            _separable_conv( 512,  512, 1),
            _separable_conv( 512,  512, 1),
            _separable_conv( 512, 1024, 2),
            _separable_conv(1024, 1024, 1),
            nn.AdaptiveAvgPool2d(1))
        self.logits = nn.Linear(1024, embedding_size)

    def forward(self, x):
        x = self.features(x)
#         avg_pool = self.AvgPool(x.data.size()[-self.dims:])
#         x = avg_pool(x).view(-1, 1024)
        x = self.fc(x)
        # TODO: add softmax
        return x
    

class YAMNet(MobileNetV1):
    def __init__(self, embedding_size=521):
        super().__init__(embedding_size=embedding_size, dims=2, input_channels=1)
        
    def forward(self, x):
#         x = self.spec_layer(x)
        # x = x[None, :, :, :]  # FIXME: This is a temporary workaround for a batchsize of 1.
#         x = x.view(x.size(0), 1, x.size(1), x.size(2))
        x = super().forward(x)
        return x

In [ ]:
# Copyright 2019 The TensorFlow Authors All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Core model definition of YAMNet."""

BATCHNORM_CENTER = True
BATCHNORM_SCALE = False
BATCHNORM_EPSILON = 1e-4

import csv

import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, layers

import features as features_lib
import params


def _batch_norm(name):
  def _bn_layer(layer_input):
    return layers.BatchNormalization(
      name=name,
      center=params.BATCHNORM_CENTER,
      scale=params.BATCHNORM_SCALE,
      epsilon=params.BATCHNORM_EPSILON)(layer_input)
  return _bn_layer


def _conv(name, kernel, stride, filters):
  def _conv_layer(layer_input):
    output = layers.Conv2D(name='{}/conv'.format(name),
                           filters=filters,
                           kernel_size=kernel,
                           strides=stride,
                           padding=params.CONV_PADDING,
                           use_bias=False,
                           activation=None)(layer_input)
    output = _batch_norm(name='{}/conv/bn'.format(name))(output)
    output = layers.ReLU(name='{}/relu'.format(name))(output)
    return output
  return _conv_layer


def _separable_conv(name, kernel, stride, filters):
  def _separable_conv_layer(layer_input):
    output = layers.DepthwiseConv2D(name='{}/depthwise_conv'.format(name),
                                    kernel_size=kernel,
                                    strides=stride,
                                    depth_multiplier=1,
                                    padding=params.CONV_PADDING,
                                    use_bias=False,
                                    activation=None)(layer_input)
    output = _batch_norm(name='{}/depthwise_conv/bn'.format(name))(output)
    output = layers.ReLU(name='{}/depthwise_conv/relu'.format(name))(output)
    output = layers.Conv2D(name='{}/pointwise_conv'.format(name),
                           filters=filters,
                           kernel_size=(1, 1),
                           strides=1,
                           padding=params.CONV_PADDING,
                           use_bias=False,
                           activation=None)(output)
    output = _batch_norm(name='{}/pointwise_conv/bn'.format(name))(output)
    output = layers.ReLU(name='{}/pointwise_conv/relu'.format(name))(output)
    return output
  return _separable_conv_layer


_YAMNET_LAYER_DEFS = [
    # (layer_function, kernel, stride, num_filters)
    (_conv,          [3, 3], 2,   32),
    (_separable_conv, [3, 3], 1,   64),
    (_separable_conv, [3, 3], 2,  128),
    (_separable_conv, [3, 3], 1,  128),
    (_separable_conv, [3, 3], 2,  256),
    (_separable_conv, [3, 3], 1,  256),
    (_separable_conv, [3, 3], 2,  512),
    (_separable_conv, [3, 3], 1,  512),
    (_separable_conv, [3, 3], 1,  512),
    (_separable_conv, [3, 3], 1,  512),
    (_separable_conv, [3, 3], 1,  512),
    (_separable_conv, [3, 3], 1,  512),
    (_separable_conv, [3, 3], 2, 1024),
    (_separable_conv, [3, 3], 1, 1024)
]


def yamnet(features):
  """Define the core YAMNet mode in Keras."""
    net = layers.Reshape(
        (params.PATCH_FRAMES, params.PATCH_BANDS, 1),
        input_shape=(params.PATCH_FRAMES, params.PATCH_BANDS))(features)
    
    for (i, (layer_fun, kernel, stride, filters)) in enumerate(_YAMNET_LAYER_DEFS):
        net = layer_fun('layer{}'.format(i + 1), kernel, stride, filters)(net)
        net = layers.GlobalAveragePooling2D()(net) ### nn.adaptive_pooling
    logits = layers.Dense(units=params.NUM_CLASSES, use_bias=True)(net)
    predictions = layers.Activation(
        name=params.EXAMPLE_PREDICTIONS_LAYER_NAME,
        activation=params.CLASSIFIER_ACTIVATION)(logits)
    return predictions


def yamnet_frames_model(feature_params):
  """Defines the YAMNet waveform-to-class-scores model.

  Args:
    feature_params: An object with parameter fields to control the feature
    calculation.

  Returns:
    A model accepting (1, num_samples) waveform input and emitting a
    (num_patches, num_classes) matrix of class scores per time frame as
    well as a (num_spectrogram_frames, num_mel_bins) spectrogram feature
    matrix.
  """
    waveform = layers.Input(batch_shape=(1, None))
    # Store the intermediate spectrogram features to use in visualization.
    spectrogram = features_lib.waveform_to_log_mel_spectrogram(
        tf.squeeze(waveform, axis=0), feature_params)
    patches = features_lib.spectrogram_to_patches(spectrogram, feature_params)
    predictions = yamnet(patches)
    frames_model = Model(name='yamnet_frames', 
                       inputs=waveform, outputs=[predictions, spectrogram])
    return frames_model


def class_names(class_map_csv):
  """Read the class name definition file and return a list of strings."""
  with open(class_map_csv) as csv_file:
    reader = csv.reader(csv_file)
    next(reader)   # Skip header
    return np.array([display_name for (_, _, display_name) in reader])


In [347]:
yamnet_layer_names = [layer.name for layer in yamnet.layers][64:]
bn_weight = np.array(yamnet.get_layer(yamnet_layer_names[1]).get_weights())[...]
yamnet.get_layer(yamnet_layer_names[-5]).weights

[<tf.Variable 'layer14/pointwise_conv/bn_17/beta:0' shape=(1024,) dtype=float32, numpy=
 array([-0.63999045, -0.26218942, -0.23407935, ..., -0.39446837,
        -0.5892259 , -0.8815479 ], dtype=float32)>,
 <tf.Variable 'layer14/pointwise_conv/bn_17/moving_mean:0' shape=(1024,) dtype=float32, numpy=
 array([ 0.02821536, -0.01664853, -0.00820669, ...,  0.01147989,
         0.05523686,  0.11307016], dtype=float32)>,
 <tf.Variable 'layer14/pointwise_conv/bn_17/moving_variance:0' shape=(1024,) dtype=float32, numpy=
 array([4.0580973e-05, 1.5788744e-04, 1.3648024e-04, ..., 1.5272970e-04,
        3.4562827e-04, 1.0626933e-02], dtype=float32)>]

In [361]:
import torch
model = YAMNet()

weights = yamnet.get_weights()
# This is gonna be fun...
for i, block in enumerate(model.features):
    if isinstance(block, nn.AdaptiveAvgPool2d):
        continue
    for j, operator in enumerate(block):
        if isinstance(operator, nn.ReLU):
            continue
            
        # Conv2d: weight only, bias not used
        if isinstance(operator, nn.Conv2d):
            w = weights.pop(0)
#             print(f"Conv: {w.shape}, remaining: len(weights) -> {len(weights)}")
            model.features[i][j].weight.data = torch.from_numpy(np.transpose(w))
        
        if isinstance(operator, nn.BatchNorm2d):            
            # beta, moving_mean, moving_variance
            # https://discuss.pytorch.org/t/batchnorm2d-mean-and-variance-of-pre-trained-model/21330/3
            # https://github.com/pytorch/pytorch/issues/16149

            # weight == gamma, bias == beta
            # TODO: how to set weight as non-learnable 1?
            
            # Epsilon TOOO !! set as ones.
            
            # Beta
            w = weights.pop(0)
#             print(f"beta: {w.shape}, remaining: len(weights) -> {len(weights)}")
            model.features[i][j].bias.data = torch.from_numpy(w)
            # Running mean
            w = weights.pop(0)
#             print(f"mean: {w.shape}, remaining: len(weights) -> {len(weights)}")
            model.features[i][j].running_mean.data = torch.from_numpy(w)
            # Running variance
            w = weights.pop(0)
#             print(f"vari: {w.shape}, remaining: len(weights) -> {len(weights)}")
            model.features[i][j].running_var.data = torch.from_numpy(w)

model.logits.weight.data = torch.from_numpy(np.transpose(weights.pop(0)))
model.logits.bias.data = torch.from_numpy(weights.pop(0))
if (len(weights) == 0):
    print("everything went in ok!")

everything went in ok!
